In [86]:
import numpy as np
import pywt
w_name = 'bior5.5'
simple_signal = np.array([1, 2, 3, 5, 12, 9, 8, 2, 4, 5])
coeffs_pywt = pywt.wavedec(simple_signal, w_name, level=1)
wavelet = pywt.Wavelet(w_name)
low_pass_reconstruction = wavelet.rec_lo
high_pass_reconstruction = wavelet.rec_hi
pywt.waverec(coeffs_pywt,w_name,'smooth')

/home/fanghz/.local/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 1 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [88]:
import numpy as np

# 自定义卷积函数
def custom_convolve(signal, filter):
    signal_len = len(signal)
    filter_len = len(filter)
    output_len = signal_len + filter_len - 1
    output = np.zeros(output_len)

    for i in range(signal_len):
        for j in range(filter_len):
            output[i + j] += signal[i] * filter[j]
    return output

# 上采样：在元素之间插入零
def upsample(signal, factor=2):
    upsampled = np.zeros(factor * len(signal))
    upsampled[1::factor] = signal  # 将原始信号放在每隔factor的奇数位置
    return upsampled

# IDWT实现
def idwt(approx_coeffs, detail_coeffs, wavelet_filters):
    # 从低频重构滤波器和高频重构滤波器中获取重构滤波器
    low_pass_reconstruction, high_pass_reconstruction = wavelet_filters
    
    current_signal = approx_coeffs  # 初始化为近似系数

    # 从最高层开始逐层重构
    for detail in detail_coeffs:
        # 对近似系数和细节系数进行上采样
        upsampled_approx = upsample(current_signal)
        upsampled_detail = upsample(detail)

        # 分别与低通滤波器和高通滤波器进行卷积
        recon_approx = custom_convolve(upsampled_approx, low_pass_reconstruction)
        recon_detail = custom_convolve(upsampled_detail, high_pass_reconstruction)

        # 将近似和细节部分相加，得到新的重构信号
        current_signal = recon_approx + recon_detail

    return current_signal

# 示例调用
# 这里的 coeffs_pywt 和 wavelet_filters 需要根据你的具体情况定义
reconstructed_signal_manual = idwt(coeffs_pywt[0], coeffs_pywt[1:], (low_pass_reconstruction, high_pass_reconstruction))
result = reconstructed_signal_manual[:len(reconstructed_signal_manual) - len(low_pass_reconstruction) + 2]


In [79]:
upsampled_approx = upsample(coeffs_pywt[0])
upsampled_detail = upsample(coeffs_pywt[1])

In [92]:
len(upsampled_approx)

16

In [11]:
i = 0
upsampled_approx = upsample(coeffs_pywt[0])
upsampled_detail = upsample(coeffs_pywt[1])

In [12]:
upsampled_detail

array([ 0.        ,  0.36849695,  0.        ,  0.19894607,  0.        ,
       -0.28333915,  0.        ,  1.75965908,  0.        ,  1.02842582,
        0.        ,  3.22661967,  0.        , -0.97189934,  0.        ,
        0.00761214])

In [20]:
x = np.array([ 2.46016518,0,0.        ,  2.46016518,  0.        ,  1.55396324,  0.        ,
        4.41013011])
y = np.array([0.36849695,0, 0.        ,  0.36849695,  0.        ,  0.19894607,  0.        ,
       -0.28333915])

In [21]:
sum(high_pass_reconstruction*x) + sum(low_pass_reconstruction*y)

-1.9095503321574456